# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [ ]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: @nic_stegmann


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'Netflix'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

O critério primeiramente estabelecido para definir a relevância dos tweets para a Netflix foi: "O tweet fala sobre a existência ou falta de algum produto no catálogo da Netflix?" caso a resposta para essa pergunta fosse sim, o tweet foi classificado como importante.

Esse critério foi estabelecido pois julgou-se importante para a empresa ter noção de quais produtos do catálogo as pessoas mais sentem faltam ou apreciam no catálogo, para que estes sejam valorizados conforme esta noção. Assim, os passos adiantes procuram criar o classificador naive bayes conforme este critério

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

### Preparando os tweets para o classificador

In [74]:
import numpy as np
import pandas as pd
import string
import mpmath

#lendo o dataframe
data = pd.read_excel("Netflix.xlsx")

#checando a probabilidade de um tweet ser relevante ou não
prob_relevancia = data.classificador.value_counts(True)
print(prob_relevancia)

def remove_pontuação(frase, lista_nova):
    for palavra in frase:
        for caractere in palavra:
            if caractere in pontuação:
                palavra = palavra.replace(caractere, "")
        lista_nova.append(palavra)

#criando parâmetros para retirar todas as pontuações das palavras
tweets = np.sum(data.Treinamento + " ")
tweets = tweets.split()
pontuação = list(string.punctuation)

#removendo pontuação de todas as palavras
palavras = []
remove_pontuação(tweets, palavras)
    
#criando um uma lista contendo apenas pavlaras que aparecem nos posts relevantes
t_relevantes = np.sum(data[data.classificador == "sim"].Treinamento + " ")
t_relevantes = t_relevantes.split()

#criando um uma lista contendo apenas pavlaras que aparecem nos posts não relevantes
t_nrelevantes = np.sum(data[data.classificador == "não"].Treinamento + " ")
t_nrelevantes = t_nrelevantes.split()

#removendo pontuação das palavras relevantes e não relevantes
p_relevantes = []
p_nrelevantes = []
remove_pontuação(t_relevantes, p_relevantes)
remove_pontuação(t_nrelevantes, p_nrelevantes)

#salvando em uma variável a quantidade de palavras diferentes
n_palavras = len(palavras)
n_palavrasr = len(p_relevantes)
n_palavrasnr = len(p_nrelevantes)




não    0.826667
sim    0.173333
Name: classificador, dtype: float64


### Calculando as probabilidades

In [75]:
colunaaux_rel = pd.Series(p_relevantes)
colunaaux_nrel = pd.Series(p_nrelevantes)

prob_prel = colunaaux_rel.value_counts()
prob_pnrel = colunaaux_nrel.value_counts()

prel_laplace = pd.Series((prob_prel + 1) / (n_palavrasr + n_palavras))
pnrel_laplace = pd.Series((prob_pnrel + 1) / (n_palavrasnr + n_palavras))


### Montando o classificador

In [77]:
teste = pd.read_excel("Netflix.xlsx", "Teste")

lista_teste = list(teste.Teste)
lista_tweets = []


for tweet in lista_teste:
    for caractere in tweet:
        if caractere in pontuação:
            tweet = tweet.replace(caractere, "")
    lista_tweets.append(tweet)

def Probabilidade_frase(lista_tweets, prob1, prob2, prob3):
    lista_probabilidade = []
    for tweets in lista_tweets:
        p_frase = 1
        for palavra in tweets:
            if palavra in prob1.index:
                p_frase = p_frase * prob1[palavra] * prob2
            else:
                p_frase = 1/(prob3 + n_palavras) * p_frase * prob2
        lista_probabilidade.append(p_frase)
    return lista_probabilidade

teste_r = Probabilidade_frase(lista_tweets, prob_prel, prob_relevancia[1], n_palavrasr)
teste_nr = Probabilidade_frase(lista_tweets, prob_pnrel, prob_relevancia[0], n_palavrasnr)


def Comparador(teste_r, teste_nr):
    resposta = []
    for i in range(len(teste_r)):
        if teste_r[i] > teste_nr[i]:
            resposta.append("sim")
        elif teste_r[i] < teste_nr[i]:
            resposta.append("não")
        elif teste_r[i] == teste_nr[i]:
            resposta.append("seila")
    return resposta
    
respostas = Comparador(teste_r, teste_nr)
data = {"tweets" : lista_teste, "classificador": teste.classificador, "respostas": respostas}

tabela_testes = pd.DataFrame(data, columns = ["tweets", "classificador", "respostas"])

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [78]:
verificação = []
for i in range(len(respostas)):
    if teste.classificador[i] == respostas[i]:
        if respostas[i] == "não":
            verificação.append("negativo verdadeiro")
        elif respostas[i] == "sim":
            verificação.append("positivo verdadeiro")
    
    elif teste.classificador[i] != respostas[i]:
        if respostas[i] == "não":
            verificação.append("negativo falso")
        elif respostas[i] == "sim":
            verificação.append("positivo falso")
            
            
verificaçãoS = pd.Series(verificação)
porcentagem = verificaçãoS.value_counts(True)
porcentagem

negativo verdadeiro    0.895
negativo falso         0.105
dtype: float64

## Conclusão do primeiro teste

A partir dos resultados obtidos na verificação do primeiro classificador Naive-bayes nota-se que há uma relação muito forte entre a P(relevante) ou P(irrelevante) e os resultados obtidos pelo classificador. Isso pode ser explicado conforme a relação matemática utilizada pelo classificador que é: P(relevancia|tweet) = (P(palavras|relevancia) * P(relevancia))/P(tweet). Como o critério estabelecido na primeira utilização do classificador indicava um P(relevante) muito menor do que P(não-relevância) e o programa não foi alimentado com uma base de dados suficientemente grande para gerar respostas precisas, o classificador resultou em apenas classificações negativas.

Tendo isso em mente, foi proposto um segundo critério de classificação de relevancia para os tweets: "O tweet fala sobre a existência de um filme no catálogo ou sobre a conta que utiliza para acessar os produtos da Netflix?". Tal critério foi considerado relevante pois daria a noção para a empresa sobre quantas pessoas utilizam contas próprias na netflix e quantas pessoas usam contas de outras pessoas para acessar seus produtos, para que assim fosse possível calcular qual a receita que teriam se todas essas pessoas tivessem que utilizar contas próprias, além dos benefícios já citados sobre saber quais filmes são mais citados pelas pessoas.

Assim, foi criado outro classificador, descrito abaixo:

### Preparando os tweets para o classificador

In [55]:
#lendo o dataframe
data2 = pd.read_excel("Netflix(1).xlsx")

#checando a probabilidade de um tweet ser relevante ou não
prob_relevancia2 = data2.classificador.value_counts(True)
print(prob_relevancia2)


#criando parâmetros para retirar todas as pontuações das palavras
tweets2 = np.sum(data2.Treinamento + " ")
tweets22 = tweets2.split()
pontuação2 = list(string.punctuation)

#removendo pontuação de todas as palavras
palavras2 = []
remove_pontuação(tweets2, palavras2)
    
#criando um uma lista contendo apenas pavlaras que aparecem nos posts relevantes
t_relevantes2 = np.sum(data2[data2.classificador == "sim"].Treinamento + " ")
t_relevantes2 = t_relevantes2.split()

#criando um uma lista contendo apenas pavlaras que aparecem nos posts não relevantes
t_nrelevantes2 = np.sum(data2[data2.classificador == "não"].Treinamento + " ")
t_nrelevantes2 = t_nrelevantes2.split()

#removendo pontuação das palavras relevantes e não relevantes
p_relevantes2 = []
p_nrelevantes2 = []
remove_pontuação(t_relevantes2, p_relevantes2)
remove_pontuação(t_nrelevantes2, p_nrelevantes2)

#salvando em uma variável a quantidade de palavras diferentes
n_palavras2 = len(palavras2)
n_palavrasr2 = len(p_relevantes2)
n_palavrasnr2 = len(p_nrelevantes2)

não    0.646667
sim    0.353333
Name: classificador, dtype: float64


### Calculando as probabilidades

In [56]:
colunaaux_rel2 = pd.Series(p_relevantes2)
colunaaux_nrel2 = pd.Series(p_nrelevantes2)

prob_prel2 = colunaaux_rel2.value_counts()
prob_pnrel2 = colunaaux_nrel2.value_counts()

prel_laplace2 = pd.Series((prob_prel2 + 1) / (n_palavrasr2 + n_palavras2))
pnrel_laplace2 = pd.Series((prob_pnrel2 + 1) / (n_palavrasnr2 + n_palavras2))


### Montando o classificador

In [79]:
teste2 = pd.read_excel("Netflix(1).xlsx", "Teste")

lista_teste2 = list(teste2.Teste)
lista_tweets2 = []


for tweet in lista_teste:
    for caractere in tweet:
        if caractere in pontuação:
            tweet = tweet.replace(caractere, "")
    lista_tweets2.append(tweet)
    
teste_r2 = Probabilidade_frase(lista_tweets2, prob_prel2, prob_relevancia2[1], n_palavrasr2)
teste_nr2 = Probabilidade_frase(lista_tweets2, prob_pnrel2, prob_relevancia2[0], n_palavrasnr2)

respostas2 = Comparador(teste_r2, teste_nr2)
data2 = {"tweets" : lista_teste2, "classificador": teste2.classificador, "respostas": respostas2}


tabela_testes2 = pd.DataFrame(data2, columns = ["tweets", "classificador", "respostas"])

## Verificando a performance

In [80]:
verificação2 = []
for i in range(len(respostas2)):
    if teste2.classificador[i] == respostas2[i]:
        if respostas2[i] == "não":
            verificação2.append("negativo verdadeiro")
        elif respostas2[i] == "sim":
            verificação2.append("positivo verdadeiro")
    
    elif teste2.classificador[i] != respostas2[i]:
        if respostas2[i] == "não":
            verificação2.append("negativo falso")
        elif respostas2[i] == "sim":
            verificação2.append("positivo falso")
            
            
verificaçãoS2 = pd.Series(verificação2)
porcentagem2 = verificaçãoS2.value_counts(True)
porcentagem2

negativo verdadeiro    0.628141
negativo falso         0.201005
positivo falso         0.150754
positivo verdadeiro    0.020101
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Após desenvolvido o classificador naive-bayes, é de grande importância a análise de certos tópicos relacionados ao projeto.

Primeiramente, é importante perceber que o resultado da verificação do projeto está intimamente relacionado ao critério de classificação da relevância de posts. Isso se dá pois caso a probabilidade de um tweet ser relevante ou não seja muito maior do que o contrário, a probabilidade de o classificador indicar um resultado equivocado é muito grande, conforme observado na elaboração do primeiro classificador. Isso é constatado na relação matemática utilizada pelo classificador que é: P(relevancia|tweet) = (P(palavras|relevancia) * P(relevancia))/P(tweet). Nos nossos cálculos descartamos o divisor pois ele é igual para ambas categoriase portanto não é relevante na comparação.

Assim, conforme a relação expressa acima, chegamos em uma outra conclusão: Tweets contendo ironia ou dupla negação tendem a ser classificados equivocadamente. Isso se deve ao fato de que a relação matemática classifica as frases de acordo com um princípio de independência das probabilidades de cada palavra aparecerem nos tweets relevantes e irrelevantes e sendo assim, ela não leva em consideração o contexto em que são utilizados. Dessa forma, mensagens que sarcásticas tendem a ter uma discrepancia em relação a sua relevancia verdadeira.

O projeto aqui desenvolvido, entretanto, possui um grande potêncial de crescimento. Conforme já mencionado na explicação dos critérios selecionados é possível, através desta pesquisa, perceber qual a tendência do mercado em relação aos produtos no catálogo da empresa e em relação ao uso indevido de contas. A partir destes dados, é possível maximizar a receita da empresa através de mudanças estratégicas nas duas areas pesquisadas no projeto. Logo, é importante que este projeto seja desenvolvido posteriormente.

Para isso, é necessário que a base de dados seja constantemente alimentada, pois assim, o programa se tornará cada vez mais preciso. Entrtanto, isso nao deve ser feito ultilizando o classificador em novos tweets, pois desse modo o programa continuará realizando comparações com os mesmos parâmetros para estimar a probabilidade de relevância do tweet. De maneira correta, o programa deve ser alimentado com classificações com os mesmos critérios, mas realizadas por algo externo ao programa. Desse modo, conforme novas referencias forem dadas ao programa mais preciso ele se tornará, pois os padrões de frequência das palavras em tweets relevantes ou não se tornará cada vez mais claro.

O classificador aqui desenvolvido, nesse sentido, pode ser utilizado para diversos fins. Por sua capacidade de análise de padrões já decorrida previamente, ele é capaz de auxiliar um engenheiro em diversos cenários, como por exemplo ná analise de falhas no comportamento de manipuladores robóticos ou até mesmo a classificação de palavras como pertecentes as mais variadas línguas do mundo.

Para melhorar o classificador aqui montado, é possível adotar os seguintes procedimentos entre outros:
1. Considerar palavras derivadas do mesmo radical como uma só, para que suas aparições sejam consideradas mais vezes, como por exemplo as palavras gostei, gostaram, gostar.
2. Excluir palavras que não contribuem para a categorização da frase, como por exemplo, "da", "do", "a" e "o".
referencias : https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/